In [4]:
# Install required dependencies
!pip install numpy>=1.24.0 pandas>=2.0.0 matplotlib>=3.7.0

## 1. Load the Data

Read the sales data from the CSV file.

In [ ]:
import pandas as pd

# Download the sales data directly from GitHub (data branch)
url = 'https://raw.githubusercontent.com/jcanizalez/pytorch-time-series-forecasting/data/data/sales.csv'
df = pd.read_csv(url)

# Preview the first five rows
df.head()